In [2]:
import os
import tushare as ts
import pandas as pd
from datetime import datetime
from dotenv import load_dotenv

import sys
sys.path.append('..')  # 将父文件夹添加到 Python 路径中，不然找不到下面的包

from utilities.tushare_limit_up import *

以下代码是utilities.tushare_limit_up中generate_limit_up函数的展开

In [3]:
if os.getenv("ENV") != "product":
    # 在开发环境中加载 .env 文件
    load_dotenv(override=True)

# 初始化pro接口
token = os.getenv('TUSHARE_API_KEY')
pro = ts.pro_api(token)

# Generate today's date in "yyyymmdd" format
today_str = datetime.now().strftime("%Y%m%d")
cal_date = getLatestTradeData(pro, today_str)

cal_date

'20240314'

# 以下是getLimitUpData的展开
limit_list_d接口获取到的数据是收盘价等于涨停板的股票
daily中进行筛选获取涨幅大于10%的股票
两者取股票代码的并集

然后对基础的数据进行合并

In [9]:
limit_up_df = pro.limit_list_d(**{ "trade_date": cal_date, "limit_type": "U", }, fields=[ "trade_date", "ts_code", "limit_amount", "fd_amount", "first_time", "last_time", "open_times", "up_stat", "limit_times", "limit" ])
daily_df = pro.daily(**{"trade_date": cal_date}, fields=[ "ts_code", "trade_date", "open", "high", "low", "close", "pre_close", "change", "pct_chg", "vol", "amount" ])

stock_basic_df = pro.stock_basic(**{"list_status": 'L'}, fields=['ts_code','name','industry'])
daily_basic_df = pro.daily_basic(**{"trade_date": cal_date}, fields=['ts_code','trade_date','circ_mv','total_mv','turnover_rate','turnover_rate_f','pe','pb'])


In [ ]:
tenpct_df = daily_df[daily_df['pct_chg'] >= 10]

# 合并tenpct_df和limit_up_df
union_tscode = pd.concat([tenpct_df['ts_code'], limit_up_df['ts_code']]).drop_duplicates()
union_tscode_df = daily_df[daily_df['ts_code'].isin(union_tscode)]
merged_df = pd.merge(union_tscode_df, stock_basic_df, on=['ts_code'], how='left')
merged_df = pd.merge(merged_df, daily_basic_df, on=['ts_code'], how='left')
merged_df = pd.merge(merged_df, limit_up_df, on=['ts_code'], how='outer')

df = merged_df

In [3]:
# 以下是最终代码，直接可调用
df, cal_date = generate_limit_up_df()

df

,ts_code,name,industry,close,pct_chg,amount,circ_mv,total_mv,pb,pe,turnover_rate,turnover_rate_f,limit_amount,fd_amount,first_time,last_time,open_times,up_stat,limit_times
43,603580.SH,艾艾精工,塑料,20.25,9.9946,3.159798,26.461323,26.461323,6.0014,NaN,12.1163,36.6428,NaN,0.517393,93021,134009,1.0,8/8,8.0
11,002298.SZ,中电兴发,软件服务,7.00,10.0629,12.863668,44.766672,51.807763,1.2605,NaN,28.7431,29.4104,NaN,0.663342,92500,100703,5.0,5/5,5.0
7,002076.SZ,星光股份,家用电器,2.64,10.0000,0.415180,27.367300,29.280887,9.1923,118.9840,1.5171,1.7180,NaN,2.323931,92500,92500,0.0,4/4,4.0
45,603963.SH,大理药业,中成药,9.92,9.9778,2.022638,21.794240,21.794240,5.6496,NaN,9.3314,32.2487,NaN,0.202037,92500,100327,3.0,3/3,3.0
44,603879.SH,永悦科技,化工原料,6.50,9.9831,3.997396,23.357389,23.446926,5.0045,NaN,17.1730,27.8745,NaN,0.336471,92501,100804,5.0,3/3,3.0
32,600178.SH,东安动力,汽车配件,13.79,9.9681,14.254443,63.720832,65.570498,2.5750,60.4727,23.6208,48.6207,NaN,0.364654,141111,144102,1.0,2/2,2.0
6,001229.SZ,魅视科技,IT设备,39.44,10.0139,1.375849,11.597691,39.440000,4.1751,46.5175,12.2347,12.2347,NaN,0.748663,93912,93912,0.0,2/2,2.0
18,002793.SZ,罗欣药业,化学制药,5.51,9.9800,0.531468,59.835244,59.926126,2.0309,NaN,0.8882,1.4925,NaN,1.237412,92500,92500,0.0,3/5,2.0
37,603127.SH,昭衍新药,医疗保健,19.98,10.0220,1.908711,125.970329,149.827762,1.8195,13.9471,1.5152,2.7208,NaN,1.575371,92500,92500,0.0,1/1,1.0
33,600241.SH,时代万恒,电气设备,7.50,9.9707,1.274348,22.072659,22.072659,1.9769,43.0802,6.0129,11.7061,NaN,0.354981,95046,95046,0.0,3/4,1.0


In [4]:
generate_limit_up_excel(df, cal_date)

已写入路径：./涨停分析-20240314.xlsx
